# Project 3-1: Inversion and sampling using diffusion models (55 pt)

In this project, given a real image (denoted as $x_0$) as input, we will invert this image into the diffusion model to get the initial noise (denoted as $x_T$). Then starting the diffusion sampling steps from this $x_T$, we can get a reconstruction $\widetilde{x}_0$ of $x_0$. We will use DDIM (Denoising Diffusion Implicit Models)[1] as the diffusion sampling method. To perform inversion, we will perform DDIM sampling in reverse order. 

## Task 0: Before you start

Before you start, you need to understand: <br>
1. The definition of forward and backward process in diffusion models. You can check the tutorial in [2]. You should also get familiar with the notations and symbols. However, you are not required to derive the formulas; <br>
2. The definition of inversion using diffusion models; <br>
3. Stable Diffusion [3] and its architecture [4], as we will use Stable Diffusion as our backbone model; <br>
4. The diffusers library [5]. Please check the "Get started" section in the link [4] to install diffusers library. Please also go through "Tutorial" section to learn its basic usage.

[1] Song, Jiaming, Chenlin Meng, and Stefano Ermon. "Denoising diffusion implicit models." arXiv preprint arXiv:2010.02502 (2020). <br>
[2] https://lilianweng.github.io/posts/2021-07-11-diffusion-models/ <br>
[3] https://github.com/Stability-AI/stablediffusion <br>
[4] Rombach, Robin, et al. "High-resolution image synthesis with latent diffusion models." Proceedings of the IEEE/CVF conference on computer vision and pattern recognition. 2022. <br>
[5] https://huggingface.co/docs/diffusers/index

## Task 1: Get Started (2.5 pt)

Please make sure you install pytorch>=1.7, numpy, PIL, transformers and diffusers. <br>
Possible library versions: <br>
pytorch==1.12.1 <br>
transformers==4.28.1 <br>
diffusers==0.16.1

In [1]:
import torch
import numpy as np
from PIL import Image
from diffusers import DDIMScheduler
from diffusers import StableDiffusionPipeline

# Supress some unnecessary warnings when loading the CLIPTextModel
from transformers import logging
logging.set_verbosity_error()

# Set device
torch_device = "cuda:1" if torch.cuda.is_available() else "cpu"

ModuleNotFoundError: No module named 'diffusers'

In [ ]:
# Load the scheduler and model pipeline from diffusers library
scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False,
                                     set_alpha_to_one=False)
sdm_model = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", scheduler=scheduler).to(torch_device)


## Task 2: Implement DDIM method (30 pt)

Here we implement both DDIM inversion and sampling in a class called `DDIMMethod`. In the generation stage, we continuously apply formula (12) in DDIM paper [1], starting from $x_T$ and ending at $x_0$. For inversion, we apply the same formula but we start from $x_0$ and end at $x_T$. <br>
Please follow the instructions below and fill out the missing lines.

In [78]:
def load_image(image_path, height=512, width=512):
    image = Image.open(image_path).convert('RGB').resize((height, width))
    image = np.array(image)
    return image


class DDIMMethod(): 
    def __init__(self, model, num_ddim_steps=50):
        self.model = model
        self.num_ddim_steps = num_ddim_steps
        self.tokenizer = self.model.tokenizer
        self.model.scheduler.set_timesteps(self.num_ddim_steps)
        self.total_steps = len(self.model.scheduler.timesteps)  # Total number of sampling steps.
        print('Total number of steps: ', self.total_steps)
        self.prompt = None
        self.context = None
    
    # Please implement formula (12) in the DDIM paper. 
    # We set sigma_t = 0 so some terms can be ignored.
    def next_step(self, noise_pred: torch.FloatTensor, timestep: int, latent: torch.FloatTensor, mode='inversion'):
        # alpha_prod_t, alpha_prod_t_next and beta_prod_t are already calculated for you.
        # Use these variables for the TODO tasks.
        if mode == 'inversion':
            timestep, next_timestep = min(timestep - self.scheduler.config.num_train_timesteps // self.scheduler.num_inference_steps, 999), timestep
            alpha_prod_t = self.scheduler.alphas_cumprod[timestep] if timestep >= 0 else self.scheduler.final_alpha_cumprod
            alpha_prod_t_next = self.scheduler.alphas_cumprod[next_timestep]

        elif mode == 'sampling':
            prev_timestep = timestep - self.scheduler.config.num_train_timesteps // self.scheduler.num_inference_steps
            alpha_prod_t = self.scheduler.alphas_cumprod[timestep]
            alpha_prod_t_next = self.scheduler.alphas_cumprod[prev_timestep] if prev_timestep >= 0 else self.scheduler.final_alpha_cumprod
                  
        beta_prod_t = 1 - alpha_prod_t

        next_original_sample =  # TODO: 
        next_sample_direction =  # TODO:
        next_sample =  # TODO:
        return next_sample
    

    # Decode the latent to image using the VAE.
    @torch.no_grad()
    def latent2image(self, latents, return_type='np'):
        latents = 1 / 0.18215 * latents.detach()
        image = self.model.vae.decode(latents)['sample']
        if return_type == 'np':
            image = (image / 2 + 0.5).clamp(0, 1)
            image = image.cpu().permute(0, 2, 3, 1).numpy()[0]
            image = (image * 255).astype(np.uint8)
        return image

    # Encode the image to latent using the VAE.
    @torch.no_grad()
    def image2latent(self, image):
        with torch.no_grad():
            if type(image) is Image:
                image = np.array(image)
            if type(image) is torch.Tensor and image.dim() == 4:
                latents = image
            else:
                image = torch.from_numpy(image).float() / 127.5 - 1
                image = image.permute(2, 0, 1).unsqueeze(0).to(self.model.device)
                latents = self.model.vae.encode(image)['latent_dist'].mean
                latents = latents * 0.18215
        return latents

    # Prepare the conditional embeddings
    # Uncond_embeddings are the embeddings of the empty string,
    # while the cond_embeddings are the embeddings of the input prompt.
    @torch.no_grad()
    def init_prompt(self, prompt: str):
        uncond_input = self.model.tokenizer(
            [""], padding="max_length", max_length=self.model.tokenizer.model_max_length,
            return_tensors="pt"
        )
        uncond_embeddings = self.model.text_encoder(uncond_input.input_ids.to(self.model.device))[0]
        text_input = self.model.tokenizer(
            [prompt],
            padding="max_length",
            max_length=self.model.tokenizer.model_max_length,
            truncation=True,
            return_tensors="pt",
        )
        text_embeddings = self.model.text_encoder(text_input.input_ids.to(self.model.device))[0]
        self.context = torch.cat([uncond_embeddings, text_embeddings])
        self.prompt = prompt

    # Please implement the DDIM Loop by continuously applying formula (12). 
    # We will perform this steps for `self.total_steps` times.
    # Note that the timesteps saved in `self.model.scheduler.timesteps` are from $T$ to $1$, not $1$ to $T$.
    @torch.no_grad()
    def ddim_loop(self, latent, mode='inversion'):
        _, cond_embeddings = self.context.chunk(2) # We only use conditional embeddings here.
        all_latents = [latent]  # We append $x_0$ to the start of the latent list.
        latent = latent.clone().detach()
        for i in range(self.total_steps):
            if mode == 'inversion':
                t = self.model.scheduler.timesteps[] # TODO: fill in the correct index.
            elif mode == 'sampling':
                t = self.model.scheduler.timesteps[] # TODO: fill in the correct index.
            noise_pred =  # TODO: refer to the diffusers to see the usage of self.model.unet. 
                          # Here we only use cond_embeddings as the encoder_hidden_states.
            latent =  # TODO: use self.next_step to get the next latent.
            all_latents.append(latent)
        return all_latents

    @property
    def scheduler(self):
        return self.model.scheduler

    @torch.no_grad()
    def invert(self, image_path: str, prompt: str):
        self.init_prompt(prompt)
        image_gt = load_image(image_path)
        latent = self.image2latent(image_gt)
        ddim_latents = self.ddim_loop(latent, mode='inversion')

        return ddim_latents
    
    @torch.no_grad()
    def sample(self, latent: torch.Tensor, prompt: str):
        self.init_prompt(prompt)
        ddim_latents = self.ddim_loop(latent, mode='sampling')
        
        return ddim_latents


## Task 3: Invert an image (5 pt)

In [ ]:
ddim_method = DDIMMethod(sdm_model)

In [87]:
# Real image inversion
image_path = "./inversion_imgs/cat_dog.jpg"
inversion_prompt = "Photo of a cat and a dog"  # prompt can be empty or a description of the image.
mode = "inversion"
ddim_latents = ddim_method.invert(image_path, inversion_prompt)

In [91]:
# Visualize 
# Please append x_0, x_10, x_20, x_30, x_40, x_50 to the image_inversion_list. Please use this order.
image_inversion_list = []
for i in range(0, 51, 10):
    image_inversion = ddim_method.latent2image()  # TODO: fill in the correct latent from ddim_latents.
    image_inversion = Image.fromarray(image_inversion)
    image_inversion_list.append(image_inversion)

In [92]:
# visualize a list of pil images
def visualize(image_list):
    width, height = image_list[0].size
    new_im = Image.new('RGB', (width * len(image_list), height))
    for i, image in enumerate(image_list):
        new_im.paste(image, (i * width, 0))
    return new_im

In [ ]:
# The images from left to right should be x_0, x_10, x_20, x_30, x_40, x_50.
visualize(image_inversion_list)

## Task 4: Test the inverted latent (7.5 pt)

We need to know that if the inverted $x_T$ we obtain is good or not. Therefore, we need to perform diffusion sampling steps to go from $x_T$ to the reconstructed $\widetilde{x}_0$, and compare $\widetilde{x}_0$ with $x_0$.

In [94]:
# We test if our reconstructed image is close to the original image.
input_latent = ddim_latents[] # TODO: fill in the correct index.
prompt = inversion_prompt # We have to use the same prompt as the one used in the inversion in order to reconstruct the image.
ddim_latents = ddim_method.sample(input_latent, prompt)

In [95]:
image_sampling_list = []
for i in range(0, 51, 10):
    image_sampling = ddim_method.latent2image()  # TODO: fill in the correct latent from ddim_latents.
    image_sampling = Image.fromarray(image_sampling)
    image_sampling_list.append(image_sampling)

In [ ]:
# The images from left to right should be x_50, x_40, x_30, x_20, x_10, x_0.
# We can see that the reconstructed image is very close to the original image.
visualize(image_sampling_list)

Here we use PSNR to evaluate the inverted image quantitatively.

In [108]:
def calculate_psnr(image1, image2):
    mse = np.mean((image1 - image2) ** 2)
    if mse - 0 < 1e-9:
        print("Same images!")
    else:
        psnr = 20 * np.log10(255.0 / np.sqrt(mse))
        print("PSNR: {}".format(psnr))

In [ ]:
# TODO: please run the cell and report the PSNR.
original_image = image_inversion_list[0]
inverted_image = image_sampling_list[-1]

calculate_psnr(np.array(inverted_image), np.array(original_image))

## Task 5: Try your own image! (10 pt)

Step 1: Invert your one own image. Follow the same procedure as above and visualize $x_{0}, x_{10}, x_{20}, x_{30}, x_{40}, x_{50}$ from left to right. Please use a prompt that describes your input image. <br>
Step 2: Reconstruct the image from the inverted latent. Follow the same procedure as above and visualize $x_{50}, x_{40}, x_{30}, x_{20}, x_{10}, x_{0}$ from left to right. Please use the same prompt as the inversion prompt.  <br>
Step 3: Compute the PSNR between your original image and the inverted image. <br>
<br>
Please keep all the output in the notebook.

In [ ]:
# TODO